In [6]:
import pandas as pd
import os
import re

In [7]:
# Load the metadata
df = pd.read_csv("spelling_info.csv")

# Path to subchapter text files
subchapter_folder = "../texts"

# Helper: Normalize subchapter name (e.g., "8j" → "08j")
def normalize_subchapter(sc):
    match = re.match(r'(\d{1,2})([a-z])', sc)
    return f"{int(match.group(1)):02d}{match.group(2)}" if match else sc

In [8]:
from collections import defaultdict

records = []
seen_subchapter_variants = set()

for _, row in df.iterrows():
    id_ = row['ID']
    official = str(row['official_name']).strip().lower()
    nahuatl = str(row['nahuatl']).strip().lower()
    typ = row['type']
    
    # Collect all possible spelling variants
    variants = set(filter(None, [official, nahuatl]))
    subchapters = []

    # Safely parse the subchapter list
    subchapter_field = str(row.get('verified subchapter', '')).strip()
    if not subchapter_field:
        continue  # Skip rows with no subchapter listed

    parts = [p.strip() for p in subchapter_field.split(';') if p.strip()]
    
    for part in parts:
        tokens = part.split()
        if len(tokens) == 1:
            subch = normalize_subchapter(tokens[0])
            subchapters.append((subch, None))
        elif len(tokens) == 2:
            variant = tokens[0].strip().lower()
            subch = normalize_subchapter(tokens[1])
            variants.add(variant)
            subchapters.append((subch, variant))

    # For each subchapter, check for any matching variant
    for subch, _ in subchapters:
        txt_path = os.path.join(subchapter_folder, f"{subch}.txt")

        # Skip missing files silently
        if not os.path.exists(txt_path):
            continue
        
        with open(txt_path, "r", encoding="utf-8") as f:
            content = f.read().lower()

        found_variants = set()
        for v in variants:
            if v in content:
                found_variants.add(v)

        if found_variants:
            for fv in sorted(found_variants):
                key = (id_, subch, fv)
                if key not in seen_subchapter_variants:
                    seen_subchapter_variants.add(key)
                    records.append({
                        "ID": id_,
                        "official_name": row['official_name'],
                        "type": typ,
                        "subchapter": subch,
                        "spelling": fv,
                        "variant_checked": fv
                    })
        else:
            key = (id_, subch, "NOT FOUND")
            if key not in seen_subchapter_variants:
                seen_subchapter_variants.add(key)
                records.append({
                    "ID": id_,
                    "official_name": row['official_name'],
                    "type": typ,
                    "subchapter": subch,
                    "spelling": "NOT FOUND",
                    "variant_checked": None
                })

# Convert to DataFrame
result_df = pd.DataFrame.from_records(records)
result_df.to_csv("verified_synonym_matches.csv", index=False)

